In [1]:
# Install Pytorch & other libraries
%pip install -q "torch==2.4.0" tensorboard pillow

# Install Hugging Face libraries
%pip install  -q --upgrade \
  "transformers==4.45.1" \
  "datasets==3.0.1" \
  "accelerate==0.34.2" \
  "evaluate==0.4.3" \
  "bitsandbytes==0.44.0" \
  "trl==0.11.1" \
  "peft==0.13.0" \
  "qwen-vl-utils"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig

# Hugging Face model id
model_id = "Qwen/Qwen2-VL-2B-Instruct"

# Load model and tokenizer
model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto"
)
processor = AutoProcessor.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
from huggingface_hub import login

login(
  token=" ", # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
adapter_path = "rishabh2003/qwen2-2b-videoqa4"

model.load_adapter(adapter_path)

adapter_config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

In [5]:
system_message = """You are a specialized video question answering assistant. Your main tasks are to analyze video content, extract relevant information,
and answer questions based on the visual data present in the GIF. You can summarize key scenes, identify objects and actions, and
provide contextual information based on the content."""

def format_test_data(sample):
    return {"test_messages" : [
          {
              "role" : "system" ,
              "content" : [{"type" : "text" , "text" : system_message}]
          },
          {
              "role" : "user",
              "content" : [
                  {
                      "type" : "video",
                      "video" : sample["URL2"]
                  },
                  {
                      "type" : "text",
                      "text" : sample['question']
                  }
              ]
          }
          ]}

In [13]:
import pyarrow
pyarrow.__version__

'14.0.2'

In [16]:
!pip install pyarrow --upgrade --use-feature=fast-deps

In [11]:
!pip install -q -U pyarrow

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [14]:
!pip install -q -U datasets==2.13.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.3 MB/s eta 0:00:00


In [17]:
import pandas as pd
from datasets import Dataset

count_csv = pd.read_csv('count.csv')
actions_csv = pd.read_csv('action.csv')
frameqa_csv = pd.read_csv('frameqa.csv')
transition_csv = pd.read_csv('transition.csv')

count_csv.shape

count_dataset = Dataset.from_pandas(count_csv)
action_dataset = Dataset.from_pandas(actions_csv)
frameqa_dataset = Dataset.from_pandas(frameqa_csv)
transition_dataset = Dataset.from_pandas(transition_csv)

count_subset = count_dataset.select(range(1000,1500))
action_subset = action_dataset.select(range(500,1000))
frameqa_subset = frameqa_dataset.select(range(500,1000))
transition_subset = transition_dataset.select(range(500,1000))

count_test_dt = [format_test_data(sample) for sample in count_subset]
action_test_dt = [format_test_data(sample) for sample in action_subset]
transition_test_dt = [format_test_data(sample) for sample in transition_subset]
frameqa_test_dt = [format_test_data(sample) for sample in frameqa_subset]

AttributeError: module 'pyarrow.lib' has no attribute 'ListViewType'

In [ ]:
from qwen_vl_utils import process_vision_info

def generate_answer(sample, model, processor):
    # Preparation for inference
    text = processor.apply_chat_template(
        sample["test_messages"], tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to(model.device)
    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=60)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    # print(output_text)
    return output_text

# let's generate the description
answer = generate_answer(count_test_dt[0], model, processor)
print(answer)